文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がiとj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．

- 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を"->"で連結して表現する
- 文節iとjに含まれる名詞句はそれぞれ，XとYに置換する

また，係り受けパスの形状は，以下の2通りが考えられる．

- 文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節jのパスを表示
- 上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を"|"で連結して表示

例えば，「吾輩はここで始めて人間というものを見た。」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

```
Xは | Yで -> 始めて -> 人間という -> ものを | 見た
Xは | Yという -> ものを | 見た
Xは | Yを | 見た
Xで -> 始めて -> Y
Xで -> 始めて -> 人間という -> Y
Xという -> Y
```

- パターン1: XとYが別経路：|で区切る、Yは名詞部分のみの置き換え
- パターン2：XとYが同じ経路：Yは文節全体を置き換え

In [1]:
import re

In [2]:
# 区切り文字
separator = re.compile('\t|,')

# 係り受け
dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
                            (-?\d+)       # 数字(係り先)
                          ''', re.VERBOSE)
NOUN_REPLACE = '@@noun@@'
SEP1 = ' -> '
SEP2 = ' | '

In [3]:
class Morph:
    def __init__(self, line):
        
        #タブとカンマで分割
        cols = separator.split(line)
        
        self.surface = cols[0] # 表層形(surface)
        self.base = cols[7]    # 基本形(base)
        self.pos = cols[1]     # 品詞(pos)
        self.pos1 = cols[2]    # 品詞細分類1(pos1)

In [4]:
class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.dst  = dst  # 係り先文節インデックス番号
        self.dsts = []
        
        self.phrase = ''
        self.phrase_noun = ''
        self.noun = False
        
        for morph in morphs:
            if morph.pos != '記号':
                self.phrase += morph.surface # 記号以外の場合文節作成
                if morph.pos == '名詞':
                    
                    # 初めての名詞の場合、名詞句として扱う
                    if self.noun == False:
                        self.noun = True
                        self.phrase_noun += NOUN_REPLACE
                        continue
                    
                    # ひとつ前が名詞で対象だった場合は表層系を置き換えない
                    elif self.noun and self.phrase_noun.endswith(NOUN_REPLACE):
                        continue
                        
                # 置換やスキップ以外の場合は、そのまま表層系を追加
                self.phrase_noun += morph.surface

In [5]:
%time

morphs = []
chunks = []
sentences = []

with open('./neko.txt.cabocha') as f:
    
    for line in f:
        dependancies = dependancy.match(line)
        
        # EOSまたは係り受け解析結果でない場合
        if not (line == 'EOS\n' or dependancies):
            morphs.append(Morph(line))
            
        # EOSまたは係り受け解析結果で、形態素解析結果がある場合
        elif len(morphs) > 0:
            chunks.append(Chunk(morphs, dst))
            morphs = []
       
        # 係り受け結果の場合
        if dependancies:
            dst = int(dependancies.group(1))
        
        # EOSで係り受け結果がある場合
        if line == 'EOS\n' and len(chunks) > 0:
            
            # 係り先を追加していく(効率化のため最終行か追加していく)
            for chunk in chunks[::-1]:
                if chunk.dst!= -1:
                    chunk.dsts.extend([chunk.dst])
                    if chunks[chunk.dst].dst != -1:
                        chunk.dsts.extend(chunks[chunk.dst].dsts)
            sentences.append(chunks)
            chunks = []

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.44 µs


In [6]:
def output_result(i, chunk, sentence):
    print('\tX:', i, chunk.phrase, chunk.dsts)
    
    # 名詞句(X)の次からYを探索するループ
    for i_y, y_chunk in enumerate(sentence[i+1:], i+1):
        
        # 名詞句の場合Yとする
        if y_chunk.noun:
            
            # Yの情報出力
            print('\t\tY:', i_y, y_chunk.phrase)
            
            result = ''
            
            # Yが経路(係り先)に含まれる場合
            if i_y in chunk.dsts:
                
                result = '\t\t\tP1:' + '\t' + sentence[i].phrase_noun.replace(NOUN_REPLACE, 'X') + SEP1
                
                # XからYまでを出力
                for i_path in chunk.dsts:
                    
                    # Yに到達したら終了
                    if i_path == i_y:
                        result += 'Y'
                        break
                    else:
                        result += sentence[i_path].phrase + SEP1
            
            # Yが経路(係り先)に含まれない場合
            else:
                result = '\t\t\tP2:' + '\t' + sentence[i].phrase_noun.replace(NOUN_REPLACE, 'X')
                
                # 積集合での最小値で共通の文節を求める
                i_goal = min(set(chunk.dsts).intersection(set(sentence[i_y].dsts)))
                
                # Xから共通文節の前、Yまで
                for i_path in chunk.dsts:                    
                    if i_path == i_goal:
                        result += SEP2 + sentence[i_y].phrase_noun.replace(NOUN_REPLACE, 'Y')
                        break
                    else:
                        result += SEP1 + sentence[i_path].phrase                 
                
                # Yの係り先から共通文節まで
                for i_path in sentence[i_y].dsts:
                    if i_path == i_goal:
                        result += SEP2 + sentence[i_goal].phrase
                    else:
                        result += SEP1 + sentence[i_path].phrase
            print(result)

In [7]:
for i_sentence, sentence in enumerate(sentences):
    
    # 文出力
    print(i_sentence, '\t', ''.join([chunk.phrase for chunk in sentence]))
    for i_chunk, chunk in enumerate(sentence):
        
        # 文節が名詞句の場合
        if chunk.noun and chunk.dst != -1:
            output_result(i_chunk, chunk, sentence)

    # 多いので制限
    if i_sentence > 5:
        break

0 	 一
1 	 吾輩は猫である
2 	 名前はまだ無い
	X: 0 名前は [2]
3 	 どこで生れたかとんと見当がつかぬ
	X: 0 どこで [1, 2, 4]
		Y: 2 かとんと
			P1:	Xで -> 生れた -> Y
		Y: 3 見当が
			P2:	Xで -> 生れた -> かとんと | Yが | つかぬ
	X: 2 かとんと [4]
		Y: 3 見当が
			P2:	Xと | Yが | つかぬ
	X: 3 見当が [4]
4 	 何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している
	X: 0 何でも [1, 5, 7]
		Y: 3 した所で
			P2:	Xでも -> 薄暗い | Yで | 泣いて -> 記憶している
		Y: 6 いた事だけは
			P2:	Xでも -> 薄暗い -> 泣いて | Yだけは | 記憶している
		Y: 7 記憶している
			P1:	Xでも -> 薄暗い -> 泣いて -> Y
	X: 3 した所で [5, 7]
		Y: 6 いた事だけは
			P2:	Xで -> 泣いて | Yだけは | 記憶している
		Y: 7 記憶している
			P1:	Xで -> 泣いて -> Y
	X: 6 いた事だけは [7]
		Y: 7 記憶している
			P1:	Xだけは -> Y
5 	 吾輩はここで始めて人間というものを見た
	X: 0 吾輩は [5]
		Y: 1 ここで
			P2:	Xは | Yで -> 始めて -> 人間という -> ものを | 見た
		Y: 3 人間という
			P2:	Xは | Yという -> ものを | 見た
		Y: 4 ものを
			P2:	Xは | Yを | 見た
	X: 1 ここで [2, 3, 4, 5]
		Y: 3 人間という
			P1:	Xで -> 始めて -> Y
		Y: 4 ものを
			P1:	Xで -> 始めて -> 人間という -> Y
	X: 3 人間という [4, 5]
		Y: 4 ものを
			P1:	Xという -> Y
	X: 4 ものを [5]
6 	 しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ
	X: 1 あとで [2, 9]
		Y: 3 それは
			P2:	Xで -> 聞くと | Y